In [162]:
import json
import os
import pandas as pd
import requests
import time
import yaml


In [2]:
# /Users/nxo/Workspace/GitHub/arcsaef/config/reporting.yaml
with open('config/reporting.yaml', 'r') as file:
        cf = yaml.safe_load(file)

In [3]:
# Highlight all journal articles of interest
# Run get_doi_ipop.js
# Save results as doi_org.txt
# /Users/nxo/Workspace/GitHub/arcsaef/data/all/doi_org.txt'
with open('data/all/doi_org.txt', mode= "r", encoding="utf-8") as f:
    doi_org_raw = f.readlines()

In [16]:
# Use doi a the key
# Use the values in ipop
doi_org = {}
for elem in doi_org_raw:
    if not elem.startswith("#"):
        if len(elem.split(":") ) == 2:
            doi_org[elem.split(":")[0].strip()] = elem.split(":")[1].strip()

In [52]:
# load organisation in config file
nrpt_orgs = json.loads(cf['non-report']['organisations'])
rpt_orgs  = json.loads(cf['report']['organisations'])
nrpt_orgs.update(rpt_orgs)
nrpt_orgs_uc = {}

for e in nrpt_orgs:
    nrpt_orgs_uc[e.upper()] = nrpt_orgs.get(e)

# update organisation abbreviation with it's long name
for o in doi_org:
    chopped = doi_org.get(o).split(";")
    if( len(chopped) == 1 ):
        for x in nrpt_orgs_uc:
            if x == doi_org.get(o).upper():
                doi_org[o] = nrpt_orgs.get(x)
# handle output with >1 ipop
    else:
        org_split = []
        for chop in chopped:
            if chop.strip().upper() in nrpt_orgs_uc.keys():
                org_split.append(nrpt_orgs_uc.get(chop.strip().upper()))
        doi_org[o] = org_split

In [11]:
url = "https://api.openalex.org/works/https://doi.org/"

for doi in doi_org:
    # there is no SAEF id that can be used to retrieve research output attributed to SAEF
    # therefore it is safer to use DOI's of known (recorded in Zotero) SAEF output 
    # /Users/nxo/Workspace/GitHub/arcsaef/data/jnls_202408/
    response = requests.get(f"{url}{doi}") 

    if response.status_code == 200:
        page  = json.loads(response.content) # dictionary 
        fname = f"{'data/jnls_202408/'}{page.get('id').split('/')[3]}.json"

        with open(fname, mode= "w", encoding="utf-8") as f:
            json.dump(page, f)

    time.sleep(1) # sleep for 1 second satisfies max 10 call per sec limit


In [71]:
# /Users/nxo/Workspace/GitHub/arcsaef/data/jnls_202408/W4392715378.json
with open('data/jnls_202408/W3210882272.json', mode= "r", encoding="utf-8") as f:
    twork = json.load(f)

In [100]:
# /Users/nxo/Workspace/GitHub/arcsaef/data/jnls_202408/
collab_orgs  = {}

# find institutions of other authors (i.e. non-saef)
for file in os.listdir('data/jnls_202408/'):
    institutions = []
    with open(f"{'data/jnls_202408/'}{file}", mode= "r", encoding="utf-8") as f:
        work = json.load(f)

        if work.get('doi')[16: ] in doi_org.keys():
            # if not isinstance(doi_org.get(work.get('doi')[16: ]), list):
            for i in work.get('authorships'):
                if len(i.get('institutions')) > 0:
                    institutions.append(i.get('institutions')[0].get('display_name'))

            collab_orgs[ work.get('doi')[16: ] ] = [doi_org.get(work.get('doi')[16: ])]
            collab_orgs[ work.get('doi')[16: ] ].append(institutions)


In [268]:
res = [val for val in collab_orgs.values()]
final_collab_orgs = {}
multi_key = []

for o in res:
    key = o[0]
    val = o[1]
    if not isinstance(key, list):
        if key not in final_collab_orgs:
           final_collab_orgs[key] = ', '.join(val)
        else:
            final_collab_orgs[key] = f"{final_collab_orgs[key]}, {', '.join(val)}"

    if isinstance(key, list) and len(val) > 0:
        for k in key:
            if k in final_collab_orgs:
            # final_collab_orgs[key] = ', '.join(val)
                print(f"{key}")
                print(f"{k} :: {val}, \n")


        # else:
        #         final_collab_orgs[k] = f"{final_collab_orgs[key]}, {', '.join(val)}"
    #         if k.strip() in final_collab_orgs.keys() :
    #             print(f"{final_collab_orgs[k]}")
    #             final_collab_orgs[k] = f"{final_collab_orgs[k]}, {', '.join(val)}"
    #             print(final_collab_orgs[k])
    #         else:
    #             print(val)
    #             final_collab_orgs[k] = f"{final_collab_orgs[k]}, {', '.join(val)}"

['University of Wollongong', 'Monash University']
University of Wollongong :: ["Centre d'Etudes Biologiques de Chizé", ['Victoria University of Wellington', 'Monash University', 'Victoria University of Wellington', 'Monash University', 'University of Pisa', 'Durham University', 'Monash University', 'Berkeley Geochronology Center', 'University of Pisa', 'University of Pisa', 'Korea Polar Research Institute', 'Korea University', 'Victoria University of Wellington', 'Victoria University of Wellington', 'GNS Science', 'Berkeley Geochronology Center', 'ETH Zurich', 'ETH Zurich', 'Scottish Enterprise', 'University of Pisa', 'Scottish Enterprise', 'Purdue University West Lafayette'], ['University of Pretoria', 'University of Pretoria', 'University of Pretoria', 'Monash University', 'University of Waikato', 'University of Pretoria', 'Monash University', 'University of Pretoria', 'University of Pretoria', 'South Australian Museum', 'University of Pretoria', 'University of Johannesburg', 'Univer

In [230]:
# ",".join(res[0 ][1])
# res[7][1]
multi_key

['James Cook University',
 'Western Australia Museum',
 'University of Wollongong',
 'Monash University',
 'Monash University',
 'Berkeley Geochronology Center',
 'Monash University',
 'University of Pretoria',
 'University of Wollongong',
 'University of the Balearic Islands',
 'La Trobe University',
 'Monash University',
 'Australian Nuclear Science & Technology Organisation',
 'University of Wollongong',
 'Monash University',
 'King Juan Carlos University',
 'South Australia Museum',
 'University of Adelaide',
 'Australian Antarctic Division',
 'James Cook University',
 'Queensland University of Technology',
 'University of Sydney',
 'University of Wollongong',
 'Monash University',
 'Australian Antarctic Division',
 'Queensland University of Technology',
 'University of Wollongong',
 'Monash University',
 'Queensland University of Technology',
 'Australian Antarctic Division',
 'Monash University',
 'University of Wollongong',
 'Monash University',
 'Victoria University of Wellingt

In [269]:
collab_orgs.get(['University of Wollongong', 'Monash University'])

TypeError: unhashable type: 'list'